In [54]:
from pathlib import Path
import os
import numpy as np
import nanonispy2 as nap
import yaml
from datetime import datetime

def get_scan(file_name, units: dict = {"length": "m", "current": "A"}, default_channel_units: dict = {"X": "m", "Y": "m", "Z": "m", "Current": "A", "LI Demod 1 X": "A", "LI Demod 1 Y": "A", "LI Demod 2 X": "A", "LI Demod 2 Y": "A"}):
    if not os.path.exists(file_name):
        print(f"Error: File \"{file_name}\" does not exist.")
        return

    root, extension = os.path.splitext(file_name)
    if extension != ".sxm":
        print("Error: attempting to open a scan that is not an sxm file.")
        return

    try:
        scan_data = nap.read.Scan(file_name) # Read the scan data. scan_data is an object whose attributes contain all the data of the scan
        channels = np.array(list(scan_data.signals.keys())) # Read the channels
        scan_header = scan_data.header
        up_or_down = scan_header.get("scan_dir", "down") # Read whether the scan was recorded in the upward or downward direction
        pixels_uncropped = scan_header.get("scan_pixels", np.array([100, 100], dtype = int)) # Read the number of pixels in the scan
        scan_range_uncropped = scan_header.get("scan_range", np.array([1E-8, 1E-8], dtype = float)) # Read the size of the scan
        bias = round(float(scan_header.get("bias", 0)), 3) # Get the bias (present in the header as a string, passed more directly as a float)
        z_controller = scan_header.get("z-controller") # Extract and convert z-controller parameters
        feedback = bool(z_controller.get("on")[0]) # Bool, true or false
        setpoint_str = z_controller.get("Setpoint")[0]
        
        # Extract and convert time parameters and convert to datetime object
        rec_date = [int(element) for element in scan_data.header.get("rec_date", "00.00.1900").split(".")]
        rec_time = [int(element) for element in scan_data.header.get("rec_time", "00:00:00").split(":")]
        dt_object = datetime(rec_date[2], rec_date[1], rec_date[0], rec_time[0], rec_time[1], rec_time[2])
        
        # Compute the re-unitization factors
        # Lengths
        channel_units = default_channel_units.copy() # Initialize channel_units to the default setting
        match units.get("length", "m"):
            case "m": L_multiplication_factor = 1
            case "dm": L_multiplication_factor = 10
            case "cm": L_multiplication_factor = 100
            case "mm": L_multiplication_factor = 1E3
            case "um": L_multiplication_factor = 1E6
            case "nm": L_multiplication_factor = 1E9
            case "A": L_multiplication_factor = 1E10
            case "pm": L_multiplication_factor = 1E12
            case "fm": L_multiplication_factor = 1E15
            case _: L_multiplication_factor = 1
        if L_multiplication_factor == 1: units["length"] = "m" # Fall back to m
    
        # Current
        match units.get("current", "A"):
            case "A": I_multiplication_factor = 1
            case "dA": I_multiplication_factor = 10
            case "cA": I_multiplication_factor = 100
            case "mA": I_multiplication_factor = 1E3
            case "uA": I_multiplication_factor = 1E6
            case "nA": I_multiplication_factor = 1E9
            case "pA": I_multiplication_factor = 1E12
            case "fA": I_multiplication_factor = 1E15
            case _: I_multiplication_factor = 1
        if I_multiplication_factor == 1: units["current"] = "A" # Fall back to A
        
        # Update the unit in channel_units (which will now be different from default_channel_units)
        length_channels = [key for key, value in default_channel_units.items() if value == "m"]
        current_channels = [key for key, value in default_channel_units.items() if value == "A"]
        for channel in length_channels:
            if channel in channel_units: channel_units[channel] = units.get("length", "m")
        for channel in current_channels:
            if channel in channel_units: channel_units[channel] = units.get("current", "A")
        filtered_channel_units = {str(key): channel_units[key] for key in channels if key in channel_units} # Remove channels that are not present in the scan
        channel_units = filtered_channel_units
        
        # Rescale the scan data by the multiplication factors determined in the reunitization        
        for channel in channels:
            for direction in ["forward", "backward"]:
                if channel in length_channels: scan_data.signals[channel][direction] = np.array(scan_data.signals[channel][direction] * L_multiplication_factor, dtype = float)
                elif channel in current_channels: scan_data.signals[channel][direction] = np.array(scan_data.signals[channel][direction] * I_multiplication_factor, dtype = float)
        
        # Stack the forward and backward scans for each channel in a tensor. Flip the backward scan
        scan_tensor_uncropped = np.stack([np.stack((np.array(scan_data.signals[channel]["forward"], dtype = float), np.flip(np.array(scan_data.signals[channel]["backward"], dtype = float), axis = 1))) for channel in channels])
        if up_or_down == "up": scan_tensor_uncropped = np.flip(scan_tensor_uncropped, axis = 2) # Flip the scan if it recorded in the upward direction
        # scan_tensor: axis 0 = direction (0 for forward, 1 for backward); axis 1 = channel; axis 2 and 3 are x and y

        # Determine which rows should be cropped off in case the scan was not completed
        masked_array = np.isnan(scan_tensor_uncropped[0, 1]) # All channels have the same number of NaN values. The backward scan has more NaN values because the scan always starts in the forward direction.
        nan_counts = np.array([sum([int(masked_array[j, i]) for i in range(len(masked_array))]) for j in range(len(masked_array[0]))])
        good_rows = np.where(nan_counts == 0)[0]
        scan_tensor = np.array([[scan_tensor_uncropped[channel, 0, good_rows], scan_tensor_uncropped[channel, 1, good_rows]] for channel in range(len(channels))])
        
        pixels = np.asarray(np.shape(scan_tensor[0, 0])) # The number of pixels is recalculated on the basis of the scans potentially being cropped
        scan_range = np.array([scan_range_uncropped[0] * pixels[0] / pixels_uncropped[0], scan_range_uncropped[1]]) # Recalculate the size of the slow scan direction after cropping
        
        # Apply the re-unitization to various attributes in the header
        scan_range = [scan_dimension * L_multiplication_factor for scan_dimension in scan_range]
        setpoint = float(setpoint_str.split()[0]) * I_multiplication_factor

        # Add new attributes to the scan object
        setattr(scan_data, "default_channel_units", default_channel_units)
        setattr(scan_data, "channel_units", channel_units)
        setattr(scan_data, "units", units)
        setattr(scan_data, "bias", bias)
        setattr(scan_data, "channels", channels)
        setattr(scan_data, "tensor_uncropped", scan_tensor_uncropped) # Uncropped means the size of the scan before deleting the rows that were not recorded
        setattr(scan_data, "pixels_uncropped", pixels_uncropped)
        setattr(scan_data, "scan_range_uncropped", scan_range_uncropped)
        setattr(scan_data, "tensor", scan_tensor)
        setattr(scan_data, "pixels", pixels)
        setattr(scan_data, "scan_range", scan_range)
        setattr(scan_data, "feedback", feedback)
        setattr(scan_data, "setpoint", setpoint)
        setattr(scan_data, "date_time", dt_object)
    
        return scan_data

    except Exception as e:
        print(f"Error reading sxm file: {e}")

def spec_times(folder):
    dat_files = np.array([str(file) for file in Path(folder).glob("*.dat")]) # Read all the dat files

    spec_files = []
    spec_times = []

    for spec_file in dat_files:
        try:
            spec_object = nap.read.Spec(spec_file)
            [spec_date, spec_time] = spec_object.header.get("Start time").split()
    
            # Extract and convert time parameters and convert to datetime object
            rec_date = [int(element) for element in spec_date.split(".")]
            rec_time = [int(element) for element in spec_time.split(":")]
            dt_object = datetime(rec_date[2], rec_date[1], rec_date[0], rec_time[0], rec_time[1], rec_time[2])
            
            spec_times.append(dt_object)
            spec_files.append(spec_file)

        except:
            pass

    return [np.asarray(spec_files, dtype = str), np.array(spec_times)]

def get_spectrum(file_name, units: dict = {"length": "m", "current": "A"}):
    if not os.path.exists(file_name):
        print(f"Error: File \"{file_name}\" does not exist.")
        return

    root, extension = os.path.splitext(file_name)
    if extension != ".dat":
        print("Error: attempting to open a spectroscopy file that is not a dat file.")
        return

    try:
        spec_object = nap.read.Spec(file_name)
        spec_header = spec_object.header
        spec_coords = np.array([spec_header.get("X (m)", 0), spec_header.get("Y (m)", 0), spec_header.get("Z (m)", 0)], dtype = float)
        [spec_date, spec_time] = spec_header.get("Start time").split()
    
        # Extract and convert time parameters and convert to datetime object
        rec_date = [int(element) for element in spec_date.split(".")]
        rec_time = [int(element) for element in spec_time.split(":")]
        dt_object = datetime(rec_date[2], rec_date[1], rec_date[0], rec_time[0], rec_time[1], rec_time[2])
        
        channels = np.array(list(spec_object.signals.keys()), dtype = str)
        spectrum_matrix = np.array(list(spec_object.signals.values()))
        
        # Compute the re-unitization factors
        # Lengths
        match units.get("length", "m"):
            case "m": L_multiplication_factor = 1
            case "dm": L_multiplication_factor = 10
            case "cm": L_multiplication_factor = 100
            case "mm": L_multiplication_factor = 1E3
            case "um": L_multiplication_factor = 1E6
            case "nm": L_multiplication_factor = 1E9
            case "A": L_multiplication_factor = 1E10
            case "pm": L_multiplication_factor = 1E12
            case "fm": L_multiplication_factor = 1E15
            case _: L_multiplication_factor = 1
        if L_multiplication_factor == 1: units["length"] = "m" # Fall back to m
    
        # Current
        match units.get("current", "A"):
            case "A": I_multiplication_factor = 1
            case "dA": I_multiplication_factor = 10
            case "cA": I_multiplication_factor = 100
            case "mA": I_multiplication_factor = 1E3
            case "uA": I_multiplication_factor = 1E6
            case "nA": I_multiplication_factor = 1E9
            case "pA": I_multiplication_factor = 1E12
            case "fA": I_multiplication_factor = 1E15
            case _: I_multiplication_factor = 1
        if I_multiplication_factor == 1: units["current"] = "A" # Fall back to A]

        spec_coords *= L_multiplication_factor
        for channel_index in range(len(channels)):
            channel = channels[channel_index]
            if channel in ["Current (A)", "Current [bwd] (A)"]:
                spectrum_matrix[channel_index] *= I_multiplication_factor
            elif channel in ["X (m)", "Y (m)", "Z (m)"]:
                spectrum_matrix[channel_index] *= L_multiplication_factor

        # Add the new attributes to the scan object
        setattr(spec_object, "coords", spec_coords)
        setattr(spec_object, "x", float(spec_coords[0]))
        setattr(spec_object, "y", float(spec_coords[1]))
        setattr(spec_object, "z", float(spec_coords[2]))
        setattr(spec_object, "channels", channels)
        setattr(spec_object, "matrix", spectrum_matrix)
        setattr(spec_object, "date_time", dt_object)
    
        return spec_object
    
    except Exception as e:
        print(f"Error: {e}")


In [62]:
spec_dir = "C:\\Nanonis\\10182025"
spec_file = "Bias-Spectroscopy00005.dat"
spec_object = get_spectrum(spec_dir + "\\" + spec_file, units = {"length": "nm", "current": "pA"})

channels = spec_object.channels
matrix = spec_object.matrix
date_time = spec_object.date_time
coords = spec_object.coords

x_channel = "Bias calc (V)"
y_channel = "Current (A)"

if x_channel not in channels:
    print(f"The requested channel {x_channel} is not present in the selected spectrum")
else:
    x_channel_index = np.where(channels == x_channel)[0][0]

if y_channel not in channels:
    print(f"The requested channel {y_channel} is not present in the selected spectrum")
else:
    y_channel_index = np.where(channels == y_channel)[0][0]

x_data = matrix[x_channel_index]
y_data = matrix[y_channel_index]

In [ ]:
scan_object = get_scan("C:\\Data\\Peter\\WS2\\062325\\img_0049.sxm", units = {"length": "nm", "current": "A"})

print(scan_object.pixels_uncropped)
print(scan_object.pixels)
print(scan_object.scan_range)
print(scan_object.scan_range_uncropped)

In [55]:
try: # Save the scan folder to the config yaml file so it opens automatically on startup next time
    with open("C:\\Scripts\\Scanalyzer\\scanalyzer\\config.yml", "w") as file:
        yaml.safe_dump({"last_file": "C:\\Scripts\\Scanalyzer\\scanalyzer\\dummy_scan.sxm"}, file)
except Exception as error:
    print("Failed to save the scan folder to the config.yml file.")
    print(error)

In [56]:
try: # Read the last scan file from the config yaml file
    with open("C:\\Scripts\\Scanalyzer\\scanalyzer\\config.yml", "r") as file:
        config = yaml.safe_load(file)
        last_file = config.get("last_file")
except:
    print("Failed to load the last scan folder from the config.yml file.")

print(last_file)

C:\Scripts\Scanalyzer\scanalyzer\dummy_scan.sxm


In [ ]:
folder = "C:\\Nanonis\\10182025\\"
sxm_file = folder + "unnamed0001.sxm"

scan_object = get_scan(sxm_file)


spec_max_file_index = len(spec_files) - 1

In [ ]:
spec_files = np.array([str(file) for file in Path(folder).glob("*.dat")]) # Read all the dat files
spec_times = []
for spec_file in spec_files:
    spec_object = nap.read.Spec(spec_file)
    [spec_date, spec_time] = spec_object.header.get("Start time").split()

    # Extract and convert time parameters and convert to datetime object
    rec_date = [int(element) for element in spec_date.split(".")]
    rec_time = [int(element) for element in spec_time.split(":")]
    dt_object = datetime(rec_date[2], rec_date[1], rec_date[0], rec_time[0], rec_time[1], rec_time[2])
    spec_times.append(dt_object)

print(spec_files)
print(spec_times)

scan_time = scan_object.date_time
[int(spec_time > scan_time) for spec_time in spec_times]